# Analysis pipeline (cross-strategy arbitrage spreads)

This notebook implements a cross-strategy event-study + mechanism pipeline for:

- **TIPS–Treasury** (arb_2/5/10)
- **UST spot–futures** basis (2/5/10)
- **CIP** (3m; multiple currencies)
- **Equity spot–futures** basis (SPX/NDX/INDU)

Key convention: all series are transformed to a **mispricing magnitude** measure in **bps** (`y_abs_bps`), so **negative coefficients imply compression toward 0**.


In [1]:
from __future__ import annotations

import ast
import hashlib
import json
import logging
import shutil
from datetime import datetime
from pathlib import Path
import sys, os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import acorr_ljungbox
sys.path.insert(2, "../src")
if 'src' in os.getcwd():
    os.chdir(os.path.pardir)
    print(os.getcwd())
else:
    print(os.getcwd())
from slr_bucket.econometrics.event_study import add_event_time, event_study_regression, jump_estimator
from slr_bucket.io import build_data_catalog, load_any_table, resolve_dataset_path, as_daily_date, coerce_num


c:\Users\Owner\Box\Winter26\slr_bucket\notebooks


In [2]:
CONFIG = {
    # Sample window for all analysis outputs
    "sample_start": "2019-01-01",
    "sample_end": "2021-12-31",

    # Core SLR dates
    "events": ["2020-04-01", "2021-03-19", "2021-03-31"],

    # Trading-day windows / bins (in trading-day units, not calendar days)
    "windows": [20, 60],
    "event_bins": [(-60, -41), (-40, -21), (-20, -1), (0, 0), (1, 20), (21, 40), (41, 60)],
    "event_range": (-60, 60),

    # Treasury tenors to keep for Treasury-based strategies
    "tenors_required": [2, 5, 10],

    # Outcomes (dataset ids resolved under data/series/)
    "outcomes": {
        "tips_treas": {"dataset": "tips_treasury_implied_rf_2010", "pattern": "arb_", "treasury_based": True},
        "ust_spot_fut": {"dataset": "treasury_sf_output", "pattern": "Treasury_SF_", "treasury_based": True},
        "cip": {"dataset": "cip_spreads_3m_bps", "pattern": "CIP_", "treasury_based": False},
        "eq_spot_fut": {
            # "datasets": ["equity_spot_spread_SPX", "equity_spot_spread_NDX", "equity_spot_spread_INDU"],
            "datasets": ["equity_spot_spread_INDU", "equity_spot_spread_NDX", "equity_spot_spread_SPX"],
            "pattern": "spread_",
            "treasury_based": False,
        },
    },

    # Sign convention: multiply raw series by SIGN_MAP[strategy] to orient them
    # (then we use y_abs_bps = abs( oriented series ) as the main dependent variable)
    "sign_map": {
        "tips_treas": +1,
        "ust_spot_fut": -1,   # file is mostly negative; flip so deviations are mostly positive
        "cip": +1,
        "eq_spot_fut": +1,
    },

    # Controls (daily)
    "total_controls": ["VIX", "HY_OAS", "BAA10Y", "issu_7_bil", "issu_14_bil", "issu_30_bil"],
    "direct_controls": ["VIX", "HY_OAS", "BAA10Y", "issu_7_bil", "issu_14_bil", "issu_30_bil", "SOFR", "spr_tgcr", "spr_effr"],

    # "Near-zero" thresholds in bps
    "near_zero_deltas": [5.0, 10.0],

    # Inference
    "hac_lags_daily": 5,
    "hac_lags_weekly": 2,

    # choose one: "TIPS", "CIP", "EQUITY_INDU", "EQUITY_NDX", "EQUITY_SPY"
    "mode": "TIPS",

    # CIP file + columns
    "cip_path": "data/series/cip_spreads_3m_bps.csv",
    "cip_cols": ["CIP_AUD_ln","CIP_CAD_ln","CIP_CHF_ln","CIP_EUR_ln","CIP_GBP_ln","CIP_JPY_ln","CIP_NZD_ln","CIP_SEK_ln"],

    # Equity files + single column in each
    "equity_indu_path": "data/series/equity_spot_spread_INDU.csv",
    "equity_indu_col": "spread_INDU_filtered",

    "equity_ndx_path": "data/series/equity_spot_spread_NDX.csv",
    "equity_ndx_col": "spread_NDX_filtered",

    "equity_spy_path": "data/series/equity_spot_spread_SPY.csv",
    "equity_spy_col": "spread_SPY_filtered",
}

repo_root = Path.cwd().parent
cfg_hash = hashlib.sha256(json.dumps(CONFIG, sort_keys=True).encode()).hexdigest()[:12]
run_stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
run_dir = repo_root / "outputs" / "summary_pipeline" / f"{run_stamp}_{cfg_hash}"
for sub in ["figures", "tables", "data", "logs"]:
    (run_dir / sub).mkdir(parents=True, exist_ok=True)
latest_dir = repo_root / "outputs" / "summary_pipeline" / "latest"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
    handlers=[logging.FileHandler(run_dir / "logs" / "pipeline.log"), logging.StreamHandler()],
    force=True,
)
logger = logging.getLogger("summary_pipeline_multi")
run_dir


C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\2288107824.py:70: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  run_stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


WindowsPath('c:/Users/Owner/Box/Winter26/slr_bucket/outputs/summary_pipeline/20260228_203643_01492bfdea93')

In [3]:
# Helpers: unit normalization, trading-day event time, binned event-study, Stargazer patch

import sys
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def _to_bps(x: pd.Series) -> pd.Series:
    """Heuristic conversion to bps. Handles:
    - bps already (typical magnitudes: 1-500)
    - percent (e.g., 3.15 for 3.15%) -> *100
    - decimal (e.g., 0.0315 for 3.15%) -> *10000
    """
    s = pd.to_numeric(x, errors="coerce")
    med = float(np.nanmedian(np.abs(s.values))) if np.isfinite(np.nanmedian(np.abs(s.values))) else np.nan
    if not np.isfinite(med) or med == 0:
        return s

    # likely decimal (0.0005 = 5 bps) or (0.03 = 300 bps)
    if med < 0.05:
        return s * 10000.0

    # likely percent units (3.5 = 350 bps)
    if med < 20.0:
        return s * 100.0

    # already bps
    return s

def add_event_time_trading(df: pd.DataFrame, event_date: str, group_col: str = "series_id") -> pd.DataFrame:
    """Add event_time in *trading-day* index units within each series.
    event_time=0 is the nearest available trading date >= event_date, else last date if event_date beyond sample.
    """
    out = df.copy()
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out = out.dropna(subset=["date"])
    t0 = pd.Timestamp(event_date)

    def _apply(g):
        g = g.sort_values("date").copy()
        dates = g["date"].values
        # choose first date >= t0; if none, choose last
        idx0 = np.searchsorted(dates, np.datetime64(t0), side="left")
        if idx0 >= len(dates):
            idx0 = len(dates) - 1
        g["event_t0_used"] = pd.Timestamp(dates[idx0])
        g["event_time"] = np.arange(len(g), dtype=int) - int(idx0)
        return g

    return out.groupby(group_col, group_keys=False).apply(_apply)

def bin_event_time(et: pd.Series, bins: list[tuple[int,int]]) -> pd.Categorical:
    labels = [f"bin_[{a},{b}]" for a,b in bins]
    # bins are closed intervals; assign by masking
    cat = pd.Series(pd.NA, index=et.index, dtype="object")
    for (a,b),lab in zip(bins, labels):
        cat[(et>=a) & (et<=b)] = lab
    return pd.Categorical(cat, categories=labels, ordered=True)

def patch_stargazer_globals():
    """Inject pd/np into any loaded stargazer modules that forgot to import them."""
    import pandas as _pd
    import numpy as _np
    for name, mod in list(sys.modules.items()):
        if name and name.startswith("stargazer") and mod is not None:
            if not hasattr(mod, "pd"):
                setattr(mod, "pd", _pd)
            if not hasattr(mod, "np"):
                setattr(mod, "np", _np)

def sanitize_for_patsy(df: pd.DataFrame, category_cols: list[str] | None = None) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        dt = str(out[c].dtype)
        if dt in ("Int64", "Int32", "Int16", "boolean"):
            out[c] = out[c].astype("float64")
    if category_cols:
        for c in category_cols:
            if c in out.columns:
                out[c] = out[c].astype("category")
    return out

def run_pooled_jump(sub: pd.DataFrame, y_col: str, controls: list[str], fe_col: str, interact_treasury: bool = True):
    use_controls = [c for c in controls if c in sub.columns]
    x_terms = ["post"] + use_controls + [f"C({fe_col})"]
    if interact_treasury and "treasury_based" in sub.columns:
        x_terms.insert(1, "post:treasury_based")
    rhs = " + ".join(x_terms)
    reg = sub[[y_col, "post", fe_col, "treasury_based", *use_controls]].dropna().copy()
    reg = sanitize_for_patsy(reg, category_cols=[fe_col])
    res = ols(f"{y_col} ~ {rhs}", data=reg).fit()
    robust = res.get_robustcov_results(cov_type="HAC", maxlags=CONFIG["hac_lags_daily"])
    return robust, reg

def run_relief_reg(df: pd.DataFrame, y_col: str, controls: list[str], fe_col: str, interact_treasury: bool = True):
    use_controls = [c for c in controls if c in df.columns]
    x_terms = ["relief"] + use_controls + [f"C({fe_col})"]
    if interact_treasury and "treasury_based" in df.columns:
        x_terms.insert(1, "relief:treasury_based")
    rhs = " + ".join(x_terms)
    reg = df[[y_col, "relief", fe_col, "treasury_based", *use_controls]].dropna().copy()
    reg = sanitize_for_patsy(reg, category_cols=[fe_col])
    res = ols(f"{y_col} ~ {rhs}", data=reg).fit()
    robust = res.get_robustcov_results(cov_type="HAC", maxlags=CONFIG["hac_lags_daily"])
    return robust, reg


## Data map (required)

Outcomes (data/series):
- tips_treasury_implied_rf_2010.parquet  (arb_*)
- treasury_sf_output.csv                (Treasury_SF_* tenors)
- cip_spreads_3m_bps.csv                (CIP_* currencies)
- equity_spot_spread_{SPX,NDX,INDU}.csv  (spread_*)

Controls:
- preferred: data/intermediate/analysis_panel (must contain CONFIG['direct_controls'])
- fallback: FRED credit/risk + repo rates + Treasury issuance

Mechanism proxies (optional):
- primary_dealer_stats_ofr_stfm_nypd_long
- bank_exposure_y9c_agg_daily


In [4]:
catalog = build_data_catalog(repo_root / "data")
catalog.to_csv(run_dir / "data" / "data_catalog.csv", index=False)
catalog.to_parquet(run_dir / "data" / "data_catalog.parquet", index=False)
catalog.to_markdown(run_dir / "data" / "data_catalog.md", index=False)
catalog.head(20)


,path,layer,rows,columns,frequency,date_min,date_max,key_columns,join_hints
0,c:\Users\Owner\Box\Winter26\slr_bucket\data\in...,intermediate,5476,"date,spread_2y_bps,spread_5y_bps,spread_10y_bp...",daily,2010-01-04,2024-12-31,date,daily:date | keys:date | layer:intermediate
1,c:\Users\Owner\Box\Winter26\slr_bucket\data\in...,intermediate,420,"date,bid_ask_spread,pubout,n_issues",monthly,1980-01-31,2014-12-31,date,keys:date | layer:intermediate
2,c:\Users\Owner\Box\Winter26\slr_bucket\data\in...,intermediate,1209,"date,fed_assets",weekly,2002-12-18,2026-02-11,date,weekly:date | keys:date | layer:intermediate
3,c:\Users\Owner\Box\Winter26\slr_bucket\data\in...,intermediate,1209,"date,fed_treasury_holdings",weekly,2002-12-18,2026-02-11,date,weekly:date | keys:date | layer:intermediate
4,c:\Users\Owner\Box\Winter26\slr_bucket\data\in...,intermediate,751,"date,sofr,sofr_volume",daily,2019-01-02,2021-12-31,date,daily:date | keys:date | layer:intermediate
5,c:\Users\Owner\Box\Winter26\slr_bucket\data\in...,intermediate,3752,"date,spread_2y_bps,spread_5y_bps,spread_10y_bp...",daily,2010-01-04,2024-12-31,date,daily:date | keys:date | layer:intermediate
6,c:\Users\Owner\Box\Winter26\slr_bucket\data\ra...,raw,3955,"Date,AUD,CAD,CHF,EUR,GBP,JPY,NZD,SEK,USD",unknown,NaT,NaT,,layer:raw
7,c:\Users\Owner\Box\Winter26\slr_bucket\data\ra...,raw,3913,"('SPX Index', 'PX_LAST'),('SPX Index', 'IDX_ES...",unknown,NaT,NaT,,layer:raw
8,c:\Users\Owner\Box\Winter26\slr_bucket\data\ra...,raw,14,"report_date,total_assets,total_reserves,total_...",quarterly,NaT,NaT,report_date,quarterly:report_date | keys:report_date | lay...
9,c:\Users\Owner\Box\Winter26\slr_bucket\data\ra...,raw,14,"report_date,total_assets,total_reserves,total_...",quarterly,NaT,NaT,report_date,quarterly:report_date | keys:report_date | lay...


In [5]:
# Load and stack outcomes across strategies into a single long panel (bps units)

def load_tips_treas():
    path = resolve_dataset_path(CONFIG["outcomes"]["tips_treas"]["dataset"], expected_dir=repo_root / "data" / "series")
    df = load_any_table(path)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["date"] = as_daily_date(df["date"])
    cols = sorted([c for c in df.columns if c.startswith(CONFIG["outcomes"]["tips_treas"]["pattern"])],
                  key=lambda c: int(c.split("_")[1]))
    if not cols:
        raise ValueError("tips_treas: no arb_* columns found")
    long = df[["date", *cols]].melt("date", var_name="raw_name", value_name="y_raw")
    long["tenor"] = long["raw_name"].str.extract(r"arb_(\d+)").astype(float).astype("Int64")
    long["strategy"] = "tips_treas"
    long["series_id"] = long["strategy"] + "_" + long["tenor"].astype("Int64").astype(str) + "y"
    long["treasury_based"] = 1
    long["y_raw_bps"] = pd.to_numeric(long["y_raw"], errors="coerce")
    return long[["date","strategy","series_id","tenor","treasury_based","y_raw_bps"]]

def load_ust_spot_fut():
    path = resolve_dataset_path(CONFIG["outcomes"]["ust_spot_fut"]["dataset"], expected_dir=repo_root / "data" / "series")
    df = load_any_table(path)
    # handle Date column name
    date_col = "Date" if "Date" in df.columns else "date"
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df["date"] = as_daily_date(df[date_col])
    cols = [c for c in df.columns if c.startswith(CONFIG["outcomes"]["ust_spot_fut"]["pattern"])]
    if not cols:
        raise ValueError("ust_spot_fut: no Treasury_SF_* columns found")
    long = df[["date", *cols]].melt("date", var_name="raw_name", value_name="y_raw")
    long["tenor"] = long["raw_name"].str.extract(r"(\d+)Y").astype(float).astype("Int64")
    long["strategy"] = "ust_spot_fut"
    long["series_id"] = long["strategy"] + "_" + long["tenor"].astype("Int64").astype(str) + "y"
    long["treasury_based"] = 1
    long["y_raw_bps"] = _to_bps(long["y_raw"])
    return long[["date","strategy","series_id","tenor","treasury_based","y_raw_bps"]]

def load_cip():
    path = resolve_dataset_path(CONFIG["outcomes"]["cip"]["dataset"], expected_dir=repo_root / "data" / "series")
    df = load_any_table(path)
    date_col = "Date" if "Date" in df.columns else "date"
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df["date"] = as_daily_date(df[date_col])
    # cols = [c for c in df.columns if c.startswith(CONFIG["outcomes"]["cip"]["pattern"]) and c != date_col]
    cols = [
        "CIP_AUD_ln","CIP_CAD_ln","CIP_CHF_ln","CIP_EUR_ln",
        "CIP_GBP_ln","CIP_JPY_ln","CIP_NZD_ln","CIP_SEK_ln",
    ]
    missing = sorted(set(cols) - set(df.columns))
    if missing:
        raise ValueError(f"cip_spreads_3m_bps is missing columns: {missing}")

    if not cols:
        raise ValueError("cip: no CIP_* columns found")
    long = df[["date", *cols]].melt("date", var_name="raw_name", value_name="y_raw")
    # currency identifier
    long["ccy"] = long["raw_name"].str.replace("CIP_","", regex=False).str.replace("_ln","", regex=False)
    long["tenor"] = "3m"
    long["strategy"] = "cip"
    long["series_id"] = long["strategy"] + "_" + long["ccy"] + "_3m"
    long["treasury_based"] = 0
    # long["y_raw_bps"] = _to_bps(long["y_raw"])
    long["y_raw_bps"] = pd.to_numeric(long["y_raw"], errors="coerce")
    return long[["date","strategy","series_id","tenor","treasury_based","y_raw_bps"]]

def load_equity():
    out = []
    for ds in CONFIG["outcomes"]["eq_spot_fut"]["datasets"]:
        path = resolve_dataset_path(ds, expected_dir=repo_root / "data" / "series")
        df = load_any_table(path)
        date_col = "Date" if "Date" in df.columns else "date"
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
        df["date"] = as_daily_date(df[date_col])

        # # pick a spread column (prefer raw spread_*, avoid *_filtered if raw exists)
        # spread_cols = [c for c in df.columns if c.startswith(CONFIG["outcomes"]["eq_spot_fut"]["pattern"])]
        # if not spread_cols:
        #     raise ValueError(f"{ds}: no spread_* columns found")

        # # heuristic: prefer non-filtered 'spread_<INDEX>' if present
        # preferred = None
        # for c in spread_cols:
        #     if "filtered" not in c.lower():
        #         preferred = c
        #         break
        # if preferred is None:
        #     preferred = spread_cols[0]
        idx_name = ds.replace("equity_spot_spread_","")  # INDU, NDX, SPY
        preferred = f"spread_{idx_name}_filtered"
        if preferred not in df.columns:
            raise ValueError(f"{ds}: expected column '{preferred}' not found. Available: {list(df.columns)}")

        idx_name = ds.replace("equity_spot_spread_","")
        tmp = df[["date", preferred]].copy()
        tmp["strategy"] = "eq_spot_fut"
        tmp["tenor"] = "index"
        tmp["series_id"] = "eq_" + idx_name
        tmp["treasury_based"] = 0
        tmp["y_raw_bps"] = _to_bps(tmp[preferred])
        out.append(tmp[["date","strategy","series_id","tenor","treasury_based","y_raw_bps"]])
    return pd.concat(out, ignore_index=True)

# Build stacked panel
# spreads_long = pd.concat(
#     [load_tips_treas(), load_ust_spot_fut(), load_cip(), load_equity()],
#     ignore_index=True
# )
# CIP ONLY 
# spreads_long = load_cip().copy()
ACTIVE = "ust_spot_fut"  # options: "cip", "eq_INDU", "eq_NDX", "eq_SPX", "tips_treas", "ust_spot_fut"

if ACTIVE == "cip":
    spreads_long = load_cip().copy()

elif ACTIVE == "eq_INDU":
    spreads_long = load_equity().copy()
    spreads_long = spreads_long[spreads_long["series_id"].astype(str).eq("eq_INDU")]

elif ACTIVE == "eq_NDX":
    spreads_long = load_equity().copy()
    spreads_long = spreads_long[spreads_long["series_id"].astype(str).eq("eq_NDX")]

elif ACTIVE == "eq_SPX":
    spreads_long = load_equity().copy()
    spreads_long = spreads_long[spreads_long["series_id"].astype(str).eq("eq_SPX")]

elif ACTIVE == "tips_treas":
    spreads_long = load_tips_treas().copy()

elif ACTIVE == "ust_spot_fut":
    spreads_long = load_ust_spot_fut().copy()

else:
    raise ValueError(f"Unknown ACTIVE={ACTIVE}")

# Hard guard: forbid stacked-strategy runs
assert spreads_long["strategy"].nunique() == 1

# Enforce Treasury tenors
keep_tenors = set(CONFIG["tenors_required"])
is_treas = spreads_long["strategy"].isin(["tips_treas","ust_spot_fut"])
spreads_long = spreads_long[~is_treas | spreads_long["tenor"].isin(keep_tenors)].copy()

# Normalize dates and drop missing
spreads_long["date"] = pd.to_datetime(spreads_long["date"], errors="coerce")
spreads_long["date"] = as_daily_date(spreads_long["date"])
spreads_long["y_raw_bps"] = pd.to_numeric(spreads_long["y_raw_bps"], errors="coerce")
spreads_long = spreads_long.dropna(subset=["date","strategy","series_id","y_raw_bps"]).reset_index(drop=True)

# Apply sign map + compute magnitude outcome
SIGN_MAP = CONFIG["sign_map"]
spreads_long["y_bps"] = spreads_long["y_raw_bps"] * spreads_long["strategy"].map(SIGN_MAP).astype(float)
spreads_long["y_abs_bps"] = spreads_long["y_bps"].abs()

# Sample restriction (keep full stacked panel consistent throughout)
spreads_long = spreads_long[
    (spreads_long["date"] >= CONFIG["sample_start"]) & (spreads_long["date"] <= CONFIG["sample_end"])
].copy()

# Quick unit/magnitude check by strategy
chk = spreads_long.groupby("strategy")["y_abs_bps"].quantile([0.5,0.9,0.99]).unstack()
chk


,0.50,0.90,0.99
strategy,,,
ust_spot_fut,1146.455556,3370.055556,4853.906667


In [6]:
# Build daily controls

from IPython.display import display
#  (prefer intermediate analysis_panel; else raw fallback). All funding rate controls are converted to bps.

needed = set(CONFIG["direct_controls"])
controls = None

# Preferred: intermediate analysis_panel
try:
    p = resolve_dataset_path("analysis_panel", expected_dir=repo_root / "data" / "intermediate")
    panel = load_any_table(p)
    panel["date"] = pd.to_datetime(panel["date"], errors="coerce")
    panel["date"] = as_daily_date(panel["date"])
    if needed.issubset(set(panel.columns)):
        logger.info("Using controls from intermediate analysis_panel: %s", p)
        controls = panel[["date", *sorted(needed)]].copy()
except Exception as exc:  # noqa: BLE001
    logger.warning("analysis_panel unavailable/invalid (%s), using raw fallback", exc)

# Fallback: raw sources
if controls is None:
    fred = load_any_table(resolve_dataset_path("controls_vix_creditspreads_fred", expected_dir=repo_root / "data" / "raw" / "event_inputs"))
    fred["date"] = pd.to_datetime(fred["date"], errors="coerce")
    fred["date"] = as_daily_date(fred["date"])

    try:
        repo = load_any_table(resolve_dataset_path("repo_rates_combined", expected_dir=repo_root / "data" / "raw" / "event_inputs"))
    except FileNotFoundError:
        repo = load_any_table(resolve_dataset_path("repo_rates_fred", expected_dir=repo_root / "data" / "raw" / "event_inputs"))
    repo["date"] = pd.to_datetime(repo["date"], errors="coerce")
    repo["date"] = as_daily_date(repo["date"])
    repo = repo.rename(columns={"TGCR": "tgcr", "EFFR": "effr"})

    if "spr_tgcr" not in repo.columns and {"SOFR","tgcr"}.issubset(repo.columns):
        repo["spr_tgcr"] = pd.to_numeric(repo["tgcr"], errors="coerce") - pd.to_numeric(repo["SOFR"], errors="coerce")
    if "spr_effr" not in repo.columns and {"SOFR","effr"}.issubset(repo.columns):
        repo["spr_effr"] = pd.to_numeric(repo["effr"], errors="coerce") - pd.to_numeric(repo["SOFR"], errors="coerce")

    issu = load_any_table(resolve_dataset_path("treasury_issuance_by_tenor_fiscaldata", expected_dir=repo_root / "data" / "raw" / "event_inputs"))
    issu["date"] = pd.to_datetime(issu.get("issue_date"), errors="coerce")
    issu["date"] = as_daily_date(issu["date"])
    issu["tenor_bucket"] = pd.to_numeric(issu["tenor_bucket"], errors="coerce")
    issu["issuance_amount"] = pd.to_numeric(issu["issuance_amount"], errors="coerce") / 1e9
    d = issu.pivot_table(index="date", columns="tenor_bucket", values="issuance_amount", aggfunc="sum").reset_index()

    # robust renaming
    rename_map = {}
    for col in d.columns:
        if col == "date":
            continue
        try:
            v = float(col)
        except Exception:
            continue
        if abs(v - 7.0) < 1e-9:
            rename_map[col] = "issu_7_bil"
        elif abs(v - 10.0) < 1e-9:
            rename_map[col] = "issu_10_bil"
        elif abs(v - 14.0) < 1e-9:
            rename_map[col] = "issu_14_bil"
        elif abs(v - 20.0) < 1e-9:
            rename_map[col] = "issu_20_bil"
        elif abs(v - 30.0) < 1e-9:
            rename_map[col] = "issu_30_bil"
    d = d.rename(columns=rename_map)

    for c in ["issu_7_bil", "issu_14_bil", "issu_30_bil", "issu_10_bil", "issu_20_bil"]:
        if c not in d.columns:
            d[c] = 0.0
    if d["issu_14_bil"].fillna(0.0).abs().sum() == 0.0:
        d["issu_14_bil"] = d.get("issu_10_bil", 0.0) + d.get("issu_20_bil", 0.0)

    for c in ["issu_7_bil", "issu_14_bil", "issu_30_bil"]:
        d[c] = pd.to_numeric(d[c], errors="coerce").fillna(0.0)

    d = d[["date", "issu_7_bil", "issu_14_bil", "issu_30_bil"]]

    fred = fred.groupby("date", as_index=False).mean(numeric_only=True)
    repo = repo.groupby("date", as_index=False).mean(numeric_only=True)
    d    = d.groupby("date", as_index=False).sum(numeric_only=True)

    controls = fred.merge(repo, on="date", how="outer").merge(d, on="date", how="outer").sort_values("date")

    keep = ["date"] + sorted(set(CONFIG["direct_controls"]) & set(controls.columns))
    controls = controls[keep].copy()

# Coerce numeric + convert funding controls to bps
controls["date"] = pd.to_datetime(controls["date"], errors="coerce")
controls["date"] = as_daily_date(controls["date"])
controls = controls.dropna(subset=["date"])

for c in [c for c in CONFIG["direct_controls"] if c in controls.columns]:
    controls[c] = pd.to_numeric(controls[c], errors="coerce")

# funding-related controls to bps
for c in ["SOFR","spr_tgcr","spr_effr","tgcr","effr"]:
    if c in controls.columns:
        controls[c] = _to_bps(controls[c])

# ensure unique by date
if controls["date"].duplicated().any():
    controls = controls.groupby("date", as_index=False).mean(numeric_only=True)

# sample trim
controls = controls[(controls["date"] >= CONFIG["sample_start"]) & (controls["date"] <= CONFIG["sample_end"])].copy()

# Missingness report (including absent columns)
def missingness_report(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    out = []
    for c in cols:
        if c not in df.columns:
            out.append({"var": c, "present": False, "missing_share": 1.0, "n_nonmissing": 0})
        else:
            s = pd.to_numeric(df[c], errors="coerce")
            out.append({"var": c, "present": True, "missing_share": float(s.isna().mean()), "n_nonmissing": int(s.notna().sum())})
    return pd.DataFrame(out).sort_values(["present","missing_share"], ascending=[True, False])

display(missingness_report(controls, CONFIG["direct_controls"]))
controls.head()


,var,present,missing_share,n_nonmissing
8,spr_effr,False,1.000000,0
3,issu_7_bil,True,0.885535,91
4,issu_14_bil,True,0.885535,91
5,issu_30_bil,True,0.885535,91
6,SOFR,True,0.015094,783
7,spr_tgcr,True,0.015094,783
0,VIX,True,0.001258,794
1,HY_OAS,True,0.001258,794
2,BAA10Y,True,0.001258,794


,date,BAA10Y,HY_OAS,SOFR,VIX,issu_14_bil,issu_30_bil,issu_7_bil,spr_tgcr
0,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,2.45,5.35,315.0,23.22,NaN,NaN,NaN,-500.0
2,2019-01-03,2.48,5.44,270.0,25.45,NaN,NaN,NaN,0.0
3,2019-01-04,2.45,5.05,245.0,21.38,NaN,NaN,NaN,-200.0
4,2019-01-07,2.42,4.83,241.0,21.40,NaN,NaN,NaN,-300.0


In [7]:
controls

,date,BAA10Y,HY_OAS,SOFR,VIX,issu_14_bil,issu_30_bil,issu_7_bil,spr_tgcr
0,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,2.45,5.35,315.0,23.22,NaN,NaN,NaN,-500.0
2,2019-01-03,2.48,5.44,270.0,25.45,NaN,NaN,NaN,0.0
3,2019-01-04,2.45,5.05,245.0,21.38,NaN,NaN,NaN,-200.0
4,2019-01-07,2.42,4.83,241.0,21.40,NaN,NaN,NaN,-300.0
...,...,...,...,...,...,...,...,...,...
790,2021-12-27,1.86,3.02,5.0,17.68,NaN,NaN,NaN,0.0
791,2021-12-28,1.86,3.01,5.0,17.54,NaN,NaN,NaN,0.0
792,2021-12-29,1.85,3.03,5.0,16.95,NaN,NaN,NaN,0.0
793,2021-12-30,1.85,3.09,5.0,17.33,NaN,NaN,NaN,0.0


In [8]:
# Merge stacked outcomes with controls (m:1 by date); construct relief indicator; persist analysis panel

# keep only needed controls for merge
use_controls = [c for c in CONFIG["direct_controls"] if c in controls.columns]
controls_use = controls[["date", *use_controls]].copy()

# validate uniqueness
if controls_use["date"].duplicated().any():
    raise RuntimeError("controls_use is not unique by date after cleaning (expected m:1 merge).")

panel_long = spreads_long.merge(controls_use, on="date", how="left", validate="m:1")

# numeric coercions for controls
for c in use_controls:
    panel_long[c] = pd.to_numeric(panel_long[c], errors="coerce")

# Relief indicator (full-sample estimand)
panel_long["relief"] = ((panel_long["date"] >= "2020-04-01") & (panel_long["date"] <= "2021-03-31")).astype(int)

# Ensure types
panel_long["treasury_based"] = pd.to_numeric(panel_long["treasury_based"], errors="coerce").fillna(0).astype(int)
panel_long["strategy"] = panel_long["strategy"].astype("category")
# Make parquet-safe dtypes
panel_long["tenor"] = panel_long["tenor"].astype(str)
panel_long["series_id"] = panel_long["series_id"].astype(str)
panel_long["strategy"] = panel_long["strategy"].astype(str)
panel_long.to_parquet(run_dir / "data" / "panel_long.parquet", index=False)
panel_long.head()


,date,strategy,series_id,tenor,treasury_based,y_raw_bps,y_bps,y_abs_bps,VIX,HY_OAS,BAA10Y,issu_7_bil,issu_14_bil,issu_30_bil,SOFR,spr_tgcr,relief
0,2019-01-02,ust_spot_fut,ust_spot_fut_2y,2,1,2658.066667,-2658.066667,2658.066667,23.22,5.35,2.45,NaN,NaN,NaN,315.0,-500.0,0
1,2019-01-03,ust_spot_fut,ust_spot_fut_2y,2,1,2895.222222,-2895.222222,2895.222222,25.45,5.44,2.48,NaN,NaN,NaN,270.0,0.0,0
2,2019-01-04,ust_spot_fut,ust_spot_fut_2y,2,1,2895.222222,-2895.222222,2895.222222,21.38,5.05,2.45,NaN,NaN,NaN,245.0,-200.0,0
3,2019-01-07,ust_spot_fut,ust_spot_fut_2y,2,1,2702.577778,-2702.577778,2702.577778,21.40,4.83,2.42,NaN,NaN,NaN,241.0,-300.0,0
4,2019-01-08,ust_spot_fut,ust_spot_fut_2y,2,1,2431.000000,-2431.000000,2431.000000,20.47,4.65,2.39,NaN,NaN,NaN,242.0,-200.0,0


In [9]:
# Table 1 (paper-ready): Arbitrage spread levels by strategy × tenor × regime (includes near-zero shares)

regimes = {
    "pre": (pd.Timestamp("2019-01-01"), pd.Timestamp("2020-03-31")),
    "relief": (pd.Timestamp("2020-04-01"), pd.Timestamp("2021-03-31")),
    "post": (pd.Timestamp("2021-04-01"), pd.Timestamp("2021-12-31")),
}
deltas = CONFIG["near_zero_deltas"]

rows = []
# for (strategy, tenor), g in panel_long.groupby(["strategy","tenor"]):
# for series_id, g in panel_long.groupby(["series_id"]): # for eq_INDU
# for (strategy, tenor), g in panel_long.groupby(["strategy","tenor"]):
for series_id, g in panel_long.groupby("series_id", dropna=False):
    g = g.sort_values("date")
    # Derive identifiers from the group (robust for single-strategy runs)
    strategy = (g["strategy"].dropna().iloc[0]
                if "strategy" in g.columns and not g["strategy"].dropna().empty
                else "unknown")
    tenor = (g["tenor"].dropna().iloc[0]
            if "tenor" in g.columns and not g["tenor"].dropna().empty
            else "NA")
    for regime, (start, end) in regimes.items():
        sub = g[(g["date"] >= start) & (g["date"] <= end)].copy()
        y = pd.to_numeric(sub["y_bps"], errors="coerce")
        ya = pd.to_numeric(sub["y_abs_bps"], errors="coerce")
        if ya.dropna().empty:
            continue

        row = {
            "strategy": str(strategy),
            "series_id": str(series_id),
            "tenor": str(tenor),
            "regime": regime,
            "N_days": int(ya.notna().sum()),
            "mean_W": float(y.mean()),
            "median_W": float(y.median()),
            "p5_W": float(y.quantile(0.05)),
            "p95_W": float(y.quantile(0.95)),
            "mean_absW": float(ya.mean()),
        }
        for d0 in deltas:
            row[f"share_absW_le_{int(d0)}"] = float((ya <= d0).mean())
        rows.append(row)

# table1 = pd.DataFrame(rows).sort_values(["strategy","tenor","regime"])
table1 = pd.DataFrame(rows).sort_values(["strategy","series_id","tenor","regime"])
# Replace prior summary_stats.csv with this paper-ready Table 1
table1.to_csv(run_dir / "tables" / "summary_stats.csv", index=False)
table1.to_csv(run_dir / "tables" / "table1_levels_nearzero.csv", index=False)

table1.head(20)


,strategy,series_id,tenor,regime,N_days,mean_W,median_W,p5_W,p95_W,mean_absW,share_absW_le_5,share_absW_le_10
2,ust_spot_fut,ust_spot_fut_10y,10,post,192,1005.869878,891.994444,278.492778,2010.622500,1016.770573,0.000000,0.000000
0,ust_spot_fut,ust_spot_fut_10y,10,pre,314,-2153.668507,-2337.094444,-4401.122778,694.347222,2470.173107,0.000000,0.000000
1,ust_spot_fut,ust_spot_fut_10y,10,relief,251,-539.043161,-609.700000,-1603.916667,767.694444,817.458123,0.003984,0.003984
5,ust_spot_fut,ust_spot_fut_2y,2,post,192,998.707697,804.500000,-318.182778,2739.091667,1256.464294,0.005208,0.005208
3,ust_spot_fut,ust_spot_fut_2y,2,pre,314,-2540.439048,-2426.800000,-4301.139444,-958.383333,2563.329848,0.000000,0.000000
4,ust_spot_fut,ust_spot_fut_2y,2,relief,251,-578.769987,-599.622222,-1611.050000,944.900000,755.623904,0.000000,0.000000
8,ust_spot_fut,ust_spot_fut_5y,5,post,192,549.956655,527.255556,-36.631667,1276.400000,578.954225,0.000000,0.000000
6,ust_spot_fut,ust_spot_fut_5y,5,pre,314,-2629.634820,-2626.944444,-4377.473333,-664.455556,2636.795188,0.000000,0.000000
7,ust_spot_fut,ust_spot_fut_5y,5,relief,251,-625.344444,-650.488889,-1779.333333,593.300000,767.842010,0.003984,0.007968


In [10]:
# Layer 1: event-window jump regressions (trading-day event time) + pooled interactions

jump_rows = []
models = []
meta = []

import re
import numpy as np
import pandas as pd

def assign_bins(df: pd.DataFrame, bins: list[tuple[int,int]], col="event_time", outcol="bin") -> pd.DataFrame:
    """
    Assign each row to a bin label like 'bin_[a,b]' based on integer event_time.
    bins is a list of (lo, hi) inclusive endpoints.
    """
    out = df.copy()
    et = pd.to_numeric(out[col], errors="coerce")
    out[outcol] = np.nan
    for lo, hi in bins:
        m = (et >= lo) & (et <= hi)
        out.loc[m, outcol] = f"bin_[{lo},{hi}]"
    return out

lo, hi = CONFIG["event_range"]

for event in CONFIG["events"]:
    
    work = add_event_time_trading(panel_long, event, group_col="series_id")
    work = work[work["event_time"].between(lo, hi)].copy()

    for W in CONFIG["windows"]:
        subW = work[work["event_time"].between(-W, W)].copy()
        subW["post"] = (subW["event_time"] >= 0).astype(int)

        # (A) by-strategy pooled within strategy (series FE) — TOTAL vs DIRECT
        for strategy in sorted(subW["strategy"].astype(str).unique().tolist()):
            g = subW[subW["strategy"].astype(str) == strategy].copy()
            if g.empty:
                continue
            for spec, controls_set in [("TOTAL", CONFIG["total_controls"]), ("DIRECT", CONFIG["direct_controls"])]:
                robust, reg = run_pooled_jump(g, y_col="y_abs_bps", controls=controls_set, fe_col="series_id", interact_treasury=False)
                # extract post coefficient
                if "post" in robust.model.exog_names:
                    i = robust.model.exog_names.index("post")
                    jump_rows.append({
                        "event": event, "window_td": W, "strategy": strategy, "spec": spec,
                        "coef_post": float(robust.params[i]), "se_post": float(robust.bse[i]),
                        "N": int(robust.nobs), "N_dates": int(reg["post"].shape[0])  # obs count; dates handled below if needed
                    })

        # (B) FULL pooled across strategies with post × treasury_based interaction (used for Table 2)
        for spec, controls_set in [("TOTAL", CONFIG["total_controls"]), ("DIRECT", CONFIG["direct_controls"])]:
            robust_all, reg_all = run_pooled_jump(subW, y_col="y_abs_bps", controls=controls_set, fe_col="series_id", interact_treasury=True)
            models.append(robust_all)
            meta.append({
                "event": event,
                "spec": spec,
                "window": f"[-{W},+{W}] trading days",
                "funding_controls": "Yes" if spec == "DIRECT" else "No",
                "n_obs": int(robust_all.nobs),
                "n_dates": int(pd.to_datetime(reg_all.index).nunique()) if hasattr(reg_all.index, "nunique") else np.nan,
            })

jump_by_strategy = pd.DataFrame(jump_rows)
jump_by_strategy.to_csv(run_dir / "tables" / "jump_by_strategy.csv", index=False)
jump_by_strategy.head()


C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\83827688.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return out.groupby(group_col, group_keys=False).apply(_apply)
C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\83827688.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return out.groupby(group_col, group_keys=False).apply(_apply)
C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\83827688.py:51: Future

,event,window_td,strategy,spec,coef_post,se_post,N,N_dates
0,2020-04-01,20,ust_spot_fut,TOTAL,-6030.996424,3012.460362,15,15
1,2020-04-01,20,ust_spot_fut,DIRECT,50.082248,23.344478,15,15
2,2020-04-01,60,ust_spot_fut,TOTAL,-580.165895,400.486507,42,42
3,2020-04-01,60,ust_spot_fut,DIRECT,-1008.389613,855.274289,42,42
4,2021-03-19,20,ust_spot_fut,TOTAL,502.261085,56.309133,18,18


In [11]:
# Layer 1: binned event-study paths by strategy (trading-day event time) + pooled treasury-based interactions
# Robust: (i) always constructs reg with 'bin'; (ii) chooses valid pre-bin baseline; (iii) preserves impact day;
# (iv) never double-adds controls; (v) no silent 0-fill for missing bins.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from statsmodels.formula.api import ols

bins = CONFIG["event_bins"]
lo, hi = CONFIG["event_range"]

rows = []
rows_pooled = []

def _bin_mid(label: str) -> float:
    m = re.search(r"\[\s*(-?\d+)\s*,\s*(-?\d+)\s*\]", str(label))
    if not m:
        return np.nan
    a, b = int(m.group(1)), int(m.group(2))
    return 0.5 * (a + b)

def choose_ref_bin(reg: pd.DataFrame, default="bin_[-20,-1]") -> str:
    """Pick a pre-event reference bin that exists in THIS regression sample."""
    b = reg["bin"].astype(str)
    if default in set(b.unique()):
        return default

    def parse_hi(lbl: str):
        m = re.search(r"\[\s*(-?\d+)\s*,\s*(-?\d+)\s*\]", lbl)
        return int(m.group(2)) if m else None

    # candidate pre bins: upper endpoint <= -1
    cand = []
    for lbl in b.unique():
        hi_ = parse_hi(str(lbl))
        if hi_ is not None and hi_ <= -1:
            cand.append((str(lbl), int((b == lbl).sum()), hi_))

    if not cand:
        raise RuntimeError("No pre-event bins available after dropna(); cannot choose a baseline bin.")

    # most frequent, tie-breaker closest to -1
    cand.sort(key=lambda x: (-x[1], abs(x[2] - (-1))))
    return cand[0][0]

def drop_controls_that_kill_impact(df: pd.DataFrame, controls: list[str]) -> list[str]:
    """Keep only controls that do not eliminate ALL impact-bin observations (event_time==0)."""
    if "event_time" not in df.columns:
        return controls
    impact = df["event_time"] == 0
    if int(impact.sum()) == 0:
        return controls

    keep = []
    for c in controls:
        if c not in df.columns:
            continue
        # must have at least one non-missing value on impact day somewhere in the sample
        if df.loc[impact, c].notna().any():
            keep.append(c)
    return keep

for event in CONFIG["events"]:
    work = add_event_time_trading(panel_long, event, group_col="series_id")
    work = work[work["event_time"].between(lo, hi)].copy()

    # create bin labels (categorical) once, on work
    work["bin"] = bin_event_time(work["event_time"], bins)

    # -----------------------------
    # (A) pooled bin × treasury_based interactions
    # -----------------------------
    for spec, controls_set in [("TOTAL", CONFIG["total_controls"]), ("DIRECT", CONFIG["direct_controls"])]:
        use_controls = [c for c in controls_set if c in work.columns]
        use_controls = drop_controls_that_kill_impact(work, use_controls)

        base_cols = ["y_abs_bps", "bin", "series_id", "treasury_based"]
        reg_cols = base_cols + use_controls
        reg = work[reg_cols].dropna().copy()

        # Ensure bin exists and is string-labeled for parsing & baseline selection
        reg["bin"] = reg["bin"].astype(str)

        n_impact = int((reg["bin"] == "bin_[0,0]").sum())
        if n_impact == 0:
            # hard fail with diagnostics (this is the root cause of bogus 0/0SE paths)
            impact_before = int((work["event_time"] == 0).sum())
            raise RuntimeError(
                f"[{event} | pooled | {spec}] bin_[0,0] has 0 obs after dropna(). "
                f"Impact rows before dropna: {impact_before}. "
                f"Controls used: {use_controls}. "
                "This means impact day is being dropped (missing y or missing controls)."
            )

        ref_bin_local = choose_ref_bin(reg, default="bin_[-20,-1]")

        # enforce baseline as ref_bin_local (so bin_[0,0] is estimated, not omitted)
        bins_present = list(dict.fromkeys(reg["bin"].tolist()))
        bins_order = [ref_bin_local] + [b for b in bins_present if b != ref_bin_local]
        reg["bin"] = pd.Categorical(reg["bin"], categories=bins_order, ordered=True)

        reg = sanitize_for_patsy(reg, category_cols=["bin", "series_id"])

        rhs = f"C(bin, Treatment(reference='{ref_bin_local}')) + C(series_id)"
        rhs += f" + C(bin, Treatment(reference='{ref_bin_local}')):treasury_based"
        if use_controls:
            rhs += " + " + " + ".join(use_controls)

        res = ols(f"y_abs_bps ~ {rhs}", data=reg).fit()
        robust = res.get_robustcov_results(cov_type="HAC", maxlags=CONFIG["hac_lags_daily"])

        # store bin×treasury interaction terms with parsed bin label
        for name, est, se in zip(robust.model.exog_names, robust.params, robust.bse):
            if "treasury_based" in name and "C(bin" in name and "Treatment" in name:
                m = re.search(r"T\.(bin_\[.*?\])", name)
                if not m:
                    continue
                lab = m.group(1)
                rows_pooled.append({
                    "event": event,
                    "spec": spec,
                    "ref_bin": ref_bin_local,
                    "bin": lab,
                    "bin_mid": _bin_mid(lab),
                    "term": name,
                    "estimate": float(est),
                    "se": float(se),
                    "ci_low": float(est - 1.96 * se),
                    "ci_high": float(est + 1.96 * se),
                    "nobs": int(robust.nobs),
                })

    # -----------------------------
    # (B) by-strategy binned paths (TOTAL vs DIRECT overlay)
    # -----------------------------
    for strategy in sorted(work["strategy"].astype(str).unique().tolist()):
        g = work[work["strategy"].astype(str) == strategy].copy()
        if g.empty:
            continue

        series_by_spec = {}

        for spec, controls_set in [("TOTAL", CONFIG["total_controls"]), ("DIRECT", CONFIG["direct_controls"])]:
            use_controls = [c for c in controls_set if c in g.columns]
            use_controls = drop_controls_that_kill_impact(g, use_controls)

            reg_cols = ["y_abs_bps", "bin", "series_id"] + use_controls
            reg = g[reg_cols].dropna().copy()

            reg["bin"] = reg["bin"].astype(str)
            n_impact = int((reg["bin"] == "bin_[0,0]").sum())
            if n_impact == 0:
                impact_before = int((g["event_time"] == 0).sum())
                raise RuntimeError(
                    f"[{event} | {strategy} | {spec}] bin_[0,0] has 0 obs after dropna(). "
                    f"Impact rows before dropna: {impact_before}. Controls used: {use_controls}."
                )

            ref_bin_local = choose_ref_bin(reg, default="bin_[-20,-1]")

            bins_present = list(dict.fromkeys(reg["bin"].tolist()))
            bins_order = [ref_bin_local] + [b for b in bins_present if b != ref_bin_local]
            reg["bin"] = pd.Categorical(reg["bin"], categories=bins_order, ordered=True)

            reg = sanitize_for_patsy(reg, category_cols=["bin", "series_id"])

            rhs = f"C(bin, Treatment(reference='{ref_bin_local}')) + C(series_id)"
            if use_controls:
                rhs += " + " + " + ".join(use_controls)

            res = ols(f"y_abs_bps ~ {rhs}", data=reg).fit()
            robust = res.get_robustcov_results(cov_type="HAC", maxlags=CONFIG["hac_lags_daily"])

            out = []
            for name, est, se in zip(robust.model.exog_names, robust.params, robust.bse):
                if "C(bin" in name and "Treatment" in name and "series_id" not in name and "treasury_based" not in name:
                    m = re.search(r"T\.(bin_\[.*?\])", name)
                    if not m:
                        continue
                    lab = m.group(1)
                    out.append({
                        "event": event,
                        "strategy": strategy,
                        "spec": spec,
                        "ref_bin": ref_bin_local,
                        "bin": lab,
                        "bin_mid": _bin_mid(lab),
                        "estimate": float(est),
                        "se": float(se),
                        "ci_low": float(est - 1.96 * se),
                        "ci_high": float(est + 1.96 * se),
                        "nobs": int(robust.nobs),
                    })

            df_out = pd.DataFrame(out).sort_values("bin_mid")
            series_by_spec[spec] = df_out
            rows.append(df_out)

        # plot overlay
        if series_by_spec:
            fig, ax = plt.subplots(figsize=(8, 4))
            for spec, dfp in series_by_spec.items():
                if dfp.empty:
                    continue
                x = dfp["bin_mid"].to_numpy()
                ax.plot(x, dfp["estimate"], marker="o", label=spec)
                ax.fill_between(x, dfp["ci_low"], dfp["ci_high"], alpha=0.15)
            ax.axhline(0, color="black", lw=1)
            ax.axvline(0, color="black", ls="--", lw=1)
            ax.set_title(f"{strategy}: binned event study around {event} (DV=|spread|, bps)")
            ax.set_xlabel("Event time (bin midpoint, trading days)")
            ax.set_ylabel("Effect relative to pre-event ref bin (bps)")
            ax.legend()
            fig.tight_layout()
            fig.savefig(run_dir / "figures" / f"event_path_{strategy}_{event}_overlay.png", dpi=150)
            plt.close(fig)

# --- Write strategy paths (always safe) ---
event_paths = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(
    columns=["event","strategy","spec","ref_bin","bin","bin_mid","estimate","se","ci_low","ci_high","nobs"]
)
event_paths.to_csv(run_dir / "tables" / "eventstudy_paths_by_strategy.csv", index=False)

# --- Write pooled bin×treasury interactions (robust to empty/malformed extraction) ---
POOLED_COLS = ["event","spec","ref_bin","bin","bin_mid","term","estimate","se","ci_low","ci_high","nobs"]

if not rows_pooled:
    pooled_bin_int = pd.DataFrame(columns=POOLED_COLS)
else:
    pooled_bin_int = pd.DataFrame(rows_pooled)

    # ensure required columns exist even if extraction produced partial dicts
    for c in POOLED_COLS:
        if c not in pooled_bin_int.columns:
            pooled_bin_int[c] = np.nan

    # coerce sort keys
    pooled_bin_int["event"] = pooled_bin_int["event"].astype(str)
    pooled_bin_int["spec"] = pooled_bin_int["spec"].astype(str)
    pooled_bin_int["bin_mid"] = pd.to_numeric(pooled_bin_int["bin_mid"], errors="coerce")

    pooled_bin_int = pooled_bin_int.sort_values(["event","spec","bin_mid"], kind="mergesort")

pooled_bin_int.to_csv(run_dir / "tables" / "eventstudy_pooled_binXtreasury.csv", index=False)

# show something useful
display(event_paths.head())
display(pooled_bin_int.head())

C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\83827688.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return out.groupby(group_col, group_keys=False).apply(_apply)
C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\83827688.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return out.groupby(group_col, group_keys=False).apply(_apply)
C:\Users\Owner\AppData\Local\Temp\ipykernel_18176\83827688.py:51: Future

,event,strategy,spec,ref_bin,bin,bin_mid,estimate,se,ci_low,ci_high,nobs
0,2020-04-01,ust_spot_fut,TOTAL,"bin_[-20,-1]","bin_[-60,-41]",-50.5,-1671.401500,701.385315,-3046.116718,-296.686282,363
1,2020-04-01,ust_spot_fut,TOTAL,"bin_[-20,-1]","bin_[-40,-21]",-30.5,-1999.563153,662.688764,-3298.433131,-700.693175,363
2,2020-04-01,ust_spot_fut,TOTAL,"bin_[-20,-1]","bin_[0,0]",0.0,-632.225279,719.135746,-2041.731342,777.280783,363
3,2020-04-01,ust_spot_fut,TOTAL,"bin_[-20,-1]","bin_[1,20]",10.5,-391.097405,354.960443,-1086.819874,304.625064,363
4,2020-04-01,ust_spot_fut,TOTAL,"bin_[-20,-1]","bin_[21,40]",30.5,-172.722089,437.620717,-1030.458695,685.014517,363


,event,spec,ref_bin,bin,bin_mid,term,estimate,se,ci_low,ci_high,nobs


In [12]:
# Table 2: Event-window jump regressions (pooled) with clear labels

import numpy as np
import pandas as pd

# --- patch stargazer globals (pd/np missing in some versions) ---
import sys
import stargazer.stargazer as _stz
import stargazer.translators.statsmodels as _stz_sm

def patch_stargazer_globals():
    for name, mod in list(sys.modules.items()):
        if name and name.startswith("stargazer") and mod is not None:
            if not hasattr(mod, "pd"):
                setattr(mod, "pd", pd)
            if not hasattr(mod, "np"):
                setattr(mod, "np", np)

patch_stargazer_globals()
patch_stargazer_globals()

from stargazer.stargazer import Stargazer

# --------- robust selection of W=keep_W models ----------
keep_W = 60

def is_keep_window(md, keep_W=60):
    # Accept several formats in md["window"]
    w = md.get("window", "")
    if isinstance(w, (int, float)):
        return int(w) == int(keep_W)
    w = str(w)
    return (f"{keep_W}" in w) and ("trading" in w or "[" in w or "W=" in w or "window" in w or w.strip() == str(keep_W))

models_keep, meta_keep = [], []
for m, md in zip(models, meta):
    if is_keep_window(md, keep_W):
        models_keep.append(m)
        meta_keep.append(md)

# Must have at least your 3 events × (TOTAL,DIRECT)
if len(models_keep) < len(CONFIG["events"]) * 2:
    raise RuntimeError(
        f"Not enough models for Table 2 at W={keep_W}. "
        f"Have {len(models_keep)}, need at least {len(CONFIG['events'])*2}. "
        "Check meta['window'] formatting and that regressions ran."
    )

# Sort columns into: event-major, then TOTAL/DIRECT
event_order = list(CONFIG["events"])
spec_order = {"TOTAL": 0, "DIRECT": 1}

order_idx = sorted(
    range(len(models_keep)),
    key=lambda i: (
        event_order.index(meta_keep[i].get("event")) if meta_keep[i].get("event") in event_order else 999,
        spec_order.get(meta_keep[i].get("spec"), 999),
    )
)
models_keep = [models_keep[i] for i in order_idx]
meta_keep   = [meta_keep[i]   for i in order_idx]

# ---- Stargazer ----
sg = Stargazer(models_keep)
sg.title("Table 2. Event-window jump regressions (DV: |arbitrage spread|, bps)")
sg.show_degrees_of_freedom(False)

# Event grouping: 3 events × 2 columns
sg.custom_columns(
    ["2020-04-01 Entry", "2021-03-19 Ann.", "2021-03-31 Exp."],
    [2, 2, 2]
)

# Some stargazer versions do NOT support column_labels; add a line instead.
try:
    sg.column_labels(["TOTAL", "DIRECT"] * 3)
except Exception:
    sg.add_line("Spec", ["TOTAL", "DIRECT"] * 3)

sg.rename_covariates({
    "post": "Post (t ≥ t0)",
    "post:treasury_based": "Post × TreasuryBased",
    "treasury_based:post": "Post × TreasuryBased",
    "SOFR": "SOFR (bps)",
    "spr_tgcr": "TGCR–SOFR (bps)",
    "spr_effr": "EFFR–SOFR (bps)",
    "VIX": "VIX",
    "HY_OAS": "HY OAS",
    "BAA10Y": "Baa–10y",
    "issu_7_bil": "Issuance 0–7y ($bn)",
    "issu_14_bil": "Issuance 7–14y ($bn)",
    "issu_30_bil": "Issuance 14y+ ($bn)",
})

# Keep only key rows (omit FE coefficients)
keep = ["post", "post:treasury_based", "treasury_based:post",
        "VIX","HY_OAS","BAA10Y","issu_7_bil","issu_14_bil","issu_30_bil",
        "SOFR","spr_tgcr","spr_effr"]

sg.covariate_order([k for k in keep if any(k in m.model.exog_names for m in models_keep)])

# Notes/lines
sg.add_line("Window", [f"[-{keep_W},+{keep_W}] trading days"] * len(models_keep))
sg.add_line("Post definition", ["1{t ≥ t0} (includes day 0)"] * len(models_keep))
sg.add_line("Series FE", ["Yes"] * len(models_keep))
sg.add_line("Funding basis controls", [md.get("funding_controls", "") for md in meta_keep])
sg.add_line("HAC lags (daily)", [str(CONFIG.get("hac_lags_daily", CONFIG.get("hac_lags", "")))] * len(models_keep))
sg.add_line("Obs.", [str(int(md.get("n_obs", np.nan))) for md in meta_keep])

html_path = run_dir / "tables" / "table2_eventwindow_jumps.html"
html_path.write_text(sg.render_html(), encoding="utf-8")

# ---- Compact CSV of key coefficients (handle both interaction name orders) ----
rows_out = []
for m, md in zip(models_keep, meta_keep):
    names = list(m.model.exog_names)

    def grab(term):
        if term in names:
            i = names.index(term)
            return float(m.params[i]), float(m.bse[i])
        return None

    post = grab("post")
    inter = grab("post:treasury_based") or grab("treasury_based:post")

    if post is not None:
        rows_out.append({
            "event": md.get("event"),
            "spec": md.get("spec"),
            "window": md.get("window"),
            "term": "post",
            "coef": post[0],
            "se": post[1],
            "n_obs": int(md.get("n_obs", np.nan)),
        })
    if inter is not None:
        rows_out.append({
            "event": md.get("event"),
            "spec": md.get("spec"),
            "window": md.get("window"),
            "term": "post:treasury_based",
            "coef": inter[0],
            "se": inter[1],
            "n_obs": int(md.get("n_obs", np.nan)),
        })

table2_key = pd.DataFrame(rows_out)
table2_key.to_csv(run_dir / "tables" / "table2_key_coefs.csv", index=False)
table2_key

c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 10
  warnings.warn('covariance of constraints does not have full '
c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 10
  warnings.warn('c

,event,spec,window,term,coef,se,n_obs
0,2020-04-01,TOTAL,"[-60,+60] trading days",post,-290.082948,200.243254,42
1,2020-04-01,TOTAL,"[-60,+60] trading days",post:treasury_based,-290.082948,200.243254,42
2,2020-04-01,DIRECT,"[-60,+60] trading days",post,-504.194807,427.637145,42
3,2020-04-01,DIRECT,"[-60,+60] trading days",post:treasury_based,-504.194807,427.637145,42
4,2021-03-19,TOTAL,"[-60,+60] trading days",post,16.028960,74.250187,45
5,2021-03-19,TOTAL,"[-60,+60] trading days",post:treasury_based,16.028960,74.250187,45
6,2021-03-19,DIRECT,"[-60,+60] trading days",post,-44.809774,123.255579,45
7,2021-03-19,DIRECT,"[-60,+60] trading days",post:treasury_based,-44.809774,123.255579,45
8,2021-03-31,TOTAL,"[-60,+60] trading days",post,187.743331,146.401332,42
9,2021-03-31,TOTAL,"[-60,+60] trading days",post:treasury_based,187.743331,146.401332,42


In [13]:
# Table 3: Relief-period regression (full-sample indicator; answers “during the exclusion period”)

import pandas as pd

# pooled across all series; series FE; TOTAL vs DIRECT
relief_models = []
relief_meta = []
rows = []

for spec, controls_set in [("TOTAL", CONFIG["total_controls"]), ("DIRECT", CONFIG["direct_controls"])]:
    robust, reg = run_relief_reg(panel_long, y_col="y_abs_bps", controls=controls_set, fe_col="series_id", interact_treasury=True)
    relief_models.append(robust)
    relief_meta.append({"spec": spec, "funding_controls": "Yes" if spec=="DIRECT" else "No", "n_obs": int(robust.nobs)})

    for term in ["relief","relief:treasury_based"]:
        if term in robust.model.exog_names:
            i = robust.model.exog_names.index(term)
            rows.append({"spec": spec, "term": term, "coef": float(robust.params[i]), "se": float(robust.bse[i]), "N": int(robust.nobs)})

table3 = pd.DataFrame(rows)
table3.to_csv(run_dir / "tables" / "table3_relief_regression.csv", index=False)
table3


,spec,term,coef,se,N
0,TOTAL,relief,-539.197339,83.794174,273
1,TOTAL,relief:treasury_based,-539.197339,83.794174,273
2,DIRECT,relief,-136.062913,90.050280,273
3,DIRECT,relief:treasury_based,-136.062913,90.050280,273


In [14]:
# Table 4: “Near-zero” summary during relief (by strategy × tenor)

import pandas as pd
deltas = CONFIG["near_zero_deltas"]

relief = panel_long[(panel_long["date"] >= "2020-04-01") & (panel_long["date"] <= "2021-03-31")].copy()

rows = []
for (strategy, tenor), g in relief.groupby(["strategy","tenor"]):
    ya = pd.to_numeric(g["y_abs_bps"], errors="coerce")
    if ya.dropna().empty:
        continue
    row = {
        "strategy": str(strategy),
        "tenor": str(tenor),
        "N_days": int(ya.notna().sum()),
        "mean_absW": float(ya.mean()),
        "median_absW": float(ya.median()),
    }
    for d0 in deltas:
        row[f"share_absW_le_{int(d0)}"] = float((ya <= d0).mean())
    rows.append(row)

table4 = pd.DataFrame(rows).sort_values(["strategy","tenor"])
table4.to_csv(run_dir / "tables" / "table4_nearzero_relief.csv", index=False)
table4


,strategy,tenor,N_days,mean_absW,median_absW,share_absW_le_5,share_absW_le_10
0,ust_spot_fut,10,251,817.458123,680.866667,0.003984,0.003984
1,ust_spot_fut,2,251,755.623904,709.166667,0.000000,0.000000
2,ust_spot_fut,5,251,767.842010,699.255556,0.003984,0.007968


In [15]:
# Layer 2 (weekly): changes in spreads, with strategy-by-strategy and pooled panels; controls in bps

layer2_note = ""
try:
    pd_long = load_any_table(resolve_dataset_path("primary_dealer_stats_ofr_stfm_nypd_long", expected_dir=repo_root / "data" / "raw" / "event_inputs"))
    bank = load_any_table(resolve_dataset_path("bank_exposure_y9c_agg_daily", expected_dir=repo_root / "data" / "raw" / "event_inputs"))

    pd_long["date"] = pd.to_datetime(pd_long["date"], errors="coerce")
    bank["date"] = pd.to_datetime(bank["date"], errors="coerce")

    # dealer utilization proxy (weekly)
    util_w = pd_long.pivot_table(index="date", columns="mnemonic", values="value", aggfunc="mean").resample("W-FRI").mean()
    util_w["utilization_index"] = util_w.sum(axis=1, min_count=1)
    util_w["utilization_lag1w"] = util_w["utilization_index"].shift(1)

    # bank exposure proxy (weekly)
    if "agg_exempt_share" not in bank.columns:
        raise KeyError("bank_exposure_y9c_agg_daily missing 'agg_exempt_share'")
    bank_w = bank.set_index("date").resample("W-FRI").mean()[["agg_exempt_share"]]

    # weekly series panel: mean within week per series_id, then first-diff
    pl = panel_long.copy()
    pl["date"] = pd.to_datetime(pl["date"], errors="coerce")
    pl = pl.dropna(subset=["date"])
    pl = pl.set_index("date")

    y_w = (pl.groupby("series_id")["y_abs_bps"].resample("W-FRI").mean().rename("y").reset_index())
    # add strategy and treasury_based labels (time-invariant per series)
    labels = panel_long[["series_id","strategy","tenor","treasury_based"]].drop_duplicates()
    y_w = y_w.merge(labels, on="series_id", how="left")

    # weekly controls (levels), then differences to match ΔW design
    ctrl_cols = [c for c in CONFIG["direct_controls"] if c in pl.columns]
    c_w = pl[ctrl_cols].resample("W-FRI").mean()

    # funding controls already in bps from controls builder; keep in bps
    # build per-week panel by joining on date
    y_w = y_w.set_index("date")
    mech = y_w.join([bank_w, util_w[["utilization_lag1w"]], c_w], how="inner").dropna()

    # relief indicator (weekly, inclusive)
    mech["relief"] = ((mech.index >= "2020-04-01") & (mech.index <= "2021-03-31")).astype(int)

    # dependent variable: weekly change in |spread|
    mech["dy"] = mech.groupby("series_id")["y"].diff()

    # controls as weekly changes (to match ΔW text)
    for c in ctrl_cols:
        mech[f"d_{c}"] = mech[c].diff()

    # z-scores
    ex_std = mech["agg_exempt_share"].std()
    util_std = mech["utilization_lag1w"].std()
    mech["z_exempt"] = (mech["agg_exempt_share"] - mech["agg_exempt_share"].mean()) / (ex_std if ex_std and ex_std > 0 else 1.0)
    mech["z_util_l1"] = (mech["utilization_lag1w"] - mech["utilization_lag1w"].mean()) / (util_std if util_std and util_std > 0 else 1.0)

    mech["relief_x_exempt"] = mech["relief"] * mech["z_exempt"]
    mech["relief_x_util"] = mech["relief"] * mech["z_util_l1"]

    # pooled panel with series FE + treasury_based interactions
    base_x = ["relief","relief_x_exempt","relief_x_util"]
    xcols = base_x + [f"d_{c}" for c in ctrl_cols]
    # interactions to align with cross-strategy mechanism claim
    mech["relief_x_treas"] = mech["relief"] * mech["treasury_based"].astype(int)
    mech["relief_x_exempt_x_treas"] = mech["relief_x_exempt"] * mech["treasury_based"].astype(int)
    mech["relief_x_util_x_treas"] = mech["relief_x_util"] * mech["treasury_based"].astype(int)
    xcols_int = xcols + ["relief_x_treas","relief_x_exempt_x_treas","relief_x_util_x_treas"]

    reg = mech.dropna(subset=["dy", *xcols_int]).copy()
    reg = sanitize_for_patsy(reg, category_cols=["series_id"])

    X = sm.add_constant(reg[xcols_int], has_constant="add")
    pooled = sm.OLS(reg["dy"], X).fit(cov_type="HAC", cov_kwds={"maxlags": CONFIG["hac_lags_weekly"]})
    pooled_out = pd.DataFrame({"term": pooled.params.index, "coef": pooled.params.values, "se": pooled.bse.values})
    pooled_out.to_csv(run_dir / "tables" / "layer2_weekly_changes_pooled.csv", index=False)

    # strategy-by-strategy regressions (same xcols without treasury interactions)
    by_rows = []
    for strategy in sorted(reg["strategy"].astype(str).unique().tolist()):
        r = reg[reg["strategy"].astype(str) == strategy].copy()
        if r.shape[0] < 20:
            continue
        Xs = sm.add_constant(r[xcols], has_constant="add")
        res = sm.OLS(r["dy"], Xs).fit(cov_type="HAC", cov_kwds={"maxlags": CONFIG["hac_lags_weekly"]})
        for term in res.params.index:
            by_rows.append({"strategy": strategy, "term": term, "coef": float(res.params[term]), "se": float(res.bse[term]), "N": int(res.nobs)})

    by_out = pd.DataFrame(by_rows)
    by_out.to_csv(run_dir / "tables" / "layer2_weekly_changes_by_strategy.csv", index=False)

    layer2_note = f"Layer 2 executed. pooled_n={int(pooled.nobs)}; strategies={sorted(reg['strategy'].astype(str).unique().tolist())}"
except Exception as exc:  # noqa: BLE001
    layer2_note = f"Layer 2 skipped gracefully: {exc}"

layer2_note


"Layer 2 executed. pooled_n=156; strategies=['ust_spot_fut']"

In [16]:
# --- Stargazer LaTeX export (same models/style as HTML) ---

import sys
import pandas as pd
import numpy as np

# Patch broken stargazer installs that reference pd/np without importing them
def patch_stargazer_globals():
    for name, mod in list(sys.modules.items()):
        if name and name.startswith("stargazer") and mod is not None:
            if not hasattr(mod, "pd"):
                setattr(mod, "pd", pd)
            if not hasattr(mod, "np"):
                setattr(mod, "np", np)

patch_stargazer_globals()

from stargazer.stargazer import Stargazer

assert len(models) > 0, "No models were estimated; cannot export Stargazer LaTeX table."

tex_path = run_dir / "tables" / "regression_table.tex"

sg = Stargazer(models)
sg.title("Pooled Jump Regressions (HAC SE)")
sg.show_degrees_of_freedom(False)

# If your HTML cell uses any additional formatting calls (custom columns, covariate order, etc.),
# mirror them here as well.

# Generate LaTeX (method name varies across stargazer versions)
if hasattr(sg, "render_latex"):
    tex = sg.render_latex()
elif hasattr(sg, "render_latex_table"):
    tex = sg.render_latex_table()
else:
    raise AttributeError("Stargazer object has no LaTeX render method (render_latex / render_latex_table).")

tex_path.write_text(tex, encoding="utf-8")

tex_path

c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 7
  warnings.warn('covariance of constraints does not have full '
c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 7
  warnings.warn('covariance of constraints does not have full '
c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
c:\ProgramData\anaconda3\envs\risk\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 8
  warnings.warn('cov

WindowsPath('c:/Users/Owner/Box/Winter26/slr_bucket/outputs/summary_pipeline/20260228_203643_01492bfdea93/tables/regression_table.tex')